# Hybrid Transport Flight Optimizer

This notebook optimizes the flight of a Hybrid Electric Aircraft on a transport mission. 

In [1]:
import numpy as np
import math

from simulator import simulate, Plot
import operator
from support_funs import GetCd, GetCl, AC
from ADRpy import constraintanalysis as ca
from ADRpy import atmospheres as at
from ADRpy import unitconversions as co


First, review the aircraft parameters stored in aircraft_params.txt

In [2]:
for i in AC:
    if not type(AC[i]) == np.ndarray:
        print(i, AC[i])

emptyMass 13.0
S 1.48
parasiteDrag 0.036
AR 14.6
e 0.85
maxPitch 8.0
maxTurnRate 5.0
clSlope 0.1
clIntercept 0.25
rollingFriction 0.008
D 0.35
payloadMass 3.0
fullTankMass 1.5
ICEMass 2.8
LCV 43000000.0
maxICEThrust 58.8
maxICEPower 1864.0
batteryCapacity 220.0
batteryMass 1.315
EMMass 0.33
maxEMPower 3000.0
maxEMTorque 2.0
maxEMrps 210.0
EMEff 0.75
g 9.81
MMair 0.02897
R 8.31
P0 101325.0
rho0 1.29
T0 273.0
tLapse 0.0065
ICEMapRows 7.0
ICEMapCols 9.0
span 4.648440598738463
maxICEeff 0.1063
idealICETorque 1.922
idealICErps 113.33
idealICEPower 1368.6050572380373
maxPropEff 0.808
optJ 0.909


Generic powerplant data is also contained in the file, but must first be ACjusted to match the aircraft's power requirement. The powerplant sizing is done using the ACRpy library. First, create an atmosphere and design concept object:

In [7]:
# atmosphere object - international standard atmosphere
isa = at.Atmosphere()

# aircraft design concept object creation requires a design-brief
designbrief = {'rwyelevation_m': 0, # altitudue of the runway
               'groundrun_m': 50, # maximumm allowed take-off distance
               'climbrate_fpm': 3 * (60 / 0.3048), # required climb rate that must be achieved
               'climbspeed_kias': co.mps2kts(30), # indidcated airspeed requirement for climb
               'climbalt_m': 5000, # altitude at which the climb rate must be achieved
               'secclimbspd_kias': co.mps2kts(25), # speed at which service ceiling must be reached
               'cruisespeed_ktas': co.mps2kts(40), # cruise velocity
               'cruisealt_m': 5000, # altitude at which the cruise speed must be achieved
               'cruisethrustfact': 1, # ratio of cruise thrust to total thrust
               'servceil_m': 5000, # alt at which the max rate of climb drops to 100 ft/min
               # dummy values to prevent errors, not needed since no turns are simulated
               'turnspeed_ktas': 10,
               'stloadfactor': 1.5
              }

# aircraft design concept object creation requires a design spec
design = {'aspectratio': AC["AR"],
          'bpr': -3 # bypass ratio; -3 indicates no thrust correction (neglected for the aircraft)
         }

# aircraft design concept object creation requires a performance estimate
designperf = {'CDTO': GetCd(AC["maxPitch"], 0), # take-off coefficient of drag
              'CLTO': GetCl(AC["maxPitch"]), # take-off coefficient of lift
              'CLmaxTO': GetCl(AC["maxPitch"]), # take-off maximum coefficient of lift
              'CLmaxclean': GetCl(AC["maxPitch"]), # max lift coefficient in flight, with (non-existant) flaps retracted
              'CDminclean': AC["parasiteDrag"], # min, zero lift drag coefficient
              'etaprop': {'take-off': 0.45, 'climb': 0.75, 'cruise': 0.85,
                        'turn': 0.85, 'servceil': 0.65}, # propeller efficiencies at different flight stages
              }

# create design concept object
concept = ca.AircraftConcept(designbrief, design, designperf, isa)

The required power in different flight phases can be found using this concept object, given a wingloACing and and a take-off weight:

In [8]:
tow = AC['batteryMass'] + AC['fullTankMass'] + AC['emptyMass'] + AC['payloadMass'] + AC['EMMass'] + AC['ICEMass']
wingloading = tow * AC['g'] / AC['S']

power = concept.powerrequired(wingloading, tow, feasibleonly=False)

toPower = co.hp2kw(power['take-off'])
climbPower = co.hp2kw(power['climb'])
cruisePower = co.hp2kw(power['cruise'])

print('Take-off power:', toPower, "kW")
print('Climb power:', climbPower, "kW")
print('Cruise power:', cruisePower, 'kW')


Take-off power: 2.462163554509147 kW
Climb power: 2.7305684394512024 kW
Cruise power: 1.5666190814620538 kW


The highest power requirement is always the cruise, size the powerplant accordingly. Suppose 60% of the power requirement is to be satisfied by the ICE, the remaining 40% by the EM. Sizing is achieved by altering the torque available from each.

In [9]:
ICEPowerReq = climbPower * 0.6
EMPowerReq = climbPower * 0.4

ICEPower = 2 * math.pi * AC['ICEMaprps'][-1] * AC['ICEMapTorque'][-1]

ICEFactor = 1000 * ICEPowerReq / ICEPower

sizedICETorqueList = AC['ICEMapTorque'] * ICEFactor

EMPower = 2 * math.pi * AC['maxEMrps'] * AC['maxEMTorque']

EMFactor = 1000 * EMPowerReq / EMPower


sizedMaxEMTorque =  AC['maxEMTorque'] * EMFactor

print("The required power for the ICE is", ICEPowerReq * 1000, 
      "W \n    The maximum power achieved by the sized engine is", 2 * math.pi * sizedICETorqueList[-1] * AC["ICEMaprps"][-1])
print("The required power for the EM is", EMPowerReq * 1000,
      "W\n    The maximum power achieved by the sized motor is", 2 * math.pi * sizedMaxEMTorque * AC["maxEMrps"])

The required power for the ICE is 1638.3410636707215 W 
    The maximum power achieved by the sized engine is 1638.341063670722
The required power for the EM is 1092.227375780481 W
    The maximum power achieved by the sized motor is 1092.227375780481


Save the sized powerplant data in the dictionary, with 10% extra power since the aircraft is about to get heavier

In [10]:
AC["ICEMapTorque"] = sizedICETorqueList * 1.1
AC["maxEMTorque"] = sizedMaxEMTorque * 1.1

The mass of the powerplant must be adjusted as well. Assumes that increase in power and mass are directly proportional.

In [11]:
AC["ICEMass"] *= ICEFactor
AC["EMMass"] *= EMFactor

print("New engine mass:", AC["ICEMass"])
print("New motor mass:", AC["EMMass"])

New engine mass: 1.609082470003131
New motor mass: 0.1365833745852342
